In [2]:
import pandas as pd

# Show ALL columns
pd.set_option("display.max_columns", None)

# Show ALL rows (be careful with huge datasets)
pd.set_option("display.max_rows", None)

# Do not truncate column content
pd.set_option("display.max_colwidth", None)

# Wider display in notebook
pd.set_option("display.width", 2000)

# Load the municipios CSV
path = "data/processed/municipios_with_lat_alt.csv"
df = pd.read_csv(path)

df.head()


,municipio,lat,alt,lon
0,Abrera,41.520446,109.301308,1.902413
1,Aguilar de Segarra,41.739490,480.315002,1.631668
2,Alella,41.495287,106.062035,2.294276
3,Alpens,42.119535,873.764343,2.101673
4,"Ametlla del Vallès, L'",41.643443,232.003311,2.268344


In [4]:
df.shape


(310, 4)

In [5]:
import pandas as pd

# Remove display limits
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 2000)

# -----------------------------
# 1. Load municipios geodata
# -----------------------------
geo_path = "data/processed/municipios_with_lat_alt.csv"
df_geo = pd.read_csv(geo_path)

print("📌 Municipios geo data:")
display(df_geo.head())
print("Shape:", df_geo.shape)

# -----------------------------
# 2. Load mobility dataset
# -----------------------------
df_path = "data/processed/final_combined_with_events_2024.csv"
df = pd.read_csv(df_path)

print("\n📌 Mobility dataset:")
display(df.head())
print("Shape:", df.shape)

# -----------------------------
# 3. Join both datasets by municipio
# -----------------------------
# We assume df contains "municipio" or "municipio_origen_name"/"municipio_destino_name"
# If not, tell me the column names and I adapt it.

if "municipio" in df.columns:
    merged = df.merge(df_geo, on="municipio", how="left")
else:
    print("\n⚠️ 'municipio' column not in df — columns available:")
    print(df.columns.tolist())
    print("Tell me which column to use for the join.")

# -----------------------------
# 4. Display merged result
# -----------------------------
if 'merged' in locals():
    print("\n📌 Merged dataset preview:")
    display(merged.head())

    print("\nMerged shape:", merged.shape)

    # Check missing lat/lon
    missing_geo = merged[merged["lat"].isna() | merged["lon"].isna()]
    print("\nMissing coordinate rows:", len(missing_geo))
    display(missing_geo.head())


📌 Municipios geo data:


,municipio,lat,alt,lon
0,Abrera,41.520446,109.301308,1.902413
1,Aguilar de Segarra,41.739490,480.315002,1.631668
2,Alella,41.495287,106.062035,2.294276
3,Alpens,42.119535,873.764343,2.101673
4,"Ametlla del Vallès, L'",41.643443,232.003311,2.268344


Shape: (310, 4)


/var/folders/z2/yk2bcpgn17zcc7cpjntmvq7w0000gn/T/ipykernel_2013/3841998581.py:23: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path)



📌 Mobility dataset:


,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes,tavg,tmin,tmax,prcp,event(y/n),name,attendance
0,2024-01-01,Lunes,1,Internacional,8001,Abrera,8003,Alella,0,8.8,5.6,14.3,0.0,n,NaN,NaN
1,2024-01-01,Lunes,1,Regional,8001,Abrera,8006,Arenys de Mar,0,8.8,5.6,14.3,0.0,n,NaN,NaN
2,2024-01-01,Lunes,1,Nacional,8001,Abrera,8008,Argençola,0,8.8,5.6,14.3,0.0,n,NaN,NaN
3,2024-01-01,Lunes,1,Regional,8001,Abrera,8015,Badalona,46,8.8,5.6,14.3,0.0,n,NaN,NaN
4,2024-01-01,Lunes,1,Residente,8001,Abrera,8015,Badalona,0,8.8,5.6,14.3,0.0,n,NaN,NaN


Shape: (24623244, 16)

⚠️ 'municipio' column not in df — columns available:
['day', 'day_of_week', 'month', 'origen', 'municipio_origen', 'municipio_origen_name', 'municipio_destino', 'municipio_destino_name', 'viajes', 'tavg', 'tmin', 'tmax', 'prcp', 'event(y/n)', 'name', 'attendance']
Tell me which column to use for the join.


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24623244 entries, 0 to 24623243
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   day                     object 
 1   day_of_week             object 
 2   month                   int64  
 3   origen                  object 
 4   municipio_origen        int64  
 5   municipio_origen_name   object 
 6   municipio_destino       int64  
 7   municipio_destino_name  object 
 8   viajes                  int64  
 9   tavg                    float64
 10  tmin                    float64
 11  tmax                    float64
 12  prcp                    float64
 13  event(y/n)              object 
 14  name                    object 
 15  attendance              float64
dtypes: float64(5), int64(4), object(7)
memory usage: 2.9+ GB


In [8]:
import pandas as pd

# Rutas de entrada
path_2023 = "data/processed/final_combined_with_events_2023.csv"
path_2024 = "data/processed/final_combined_with_events_2024.csv"

# Cargar dataframes
df_2023 = pd.read_csv(path_2023)
df_2024 = pd.read_csv(path_2024)

# Unir y ordenar por la columna 'day'
df_total = pd.concat([df_2023, df_2024], ignore_index=True)
df_total["day"] = pd.to_datetime(df_total["day"])   # Asegura formato fecha
df_total = df_total.sort_values("day")

# Ruta de salida
output_path = "/Users/noelp/PycharmProjects/OPTIMET-BCN/data/processed/final_combined_2023_2024.csv"

# Guardar
df_total.to_csv(output_path, index=False)

print("CSV guardado en:", output_path)


/var/folders/z2/yk2bcpgn17zcc7cpjntmvq7w0000gn/T/ipykernel_2013/3031544960.py:8: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv(path_2023)
/var/folders/z2/yk2bcpgn17zcc7cpjntmvq7w0000gn/T/ipykernel_2013/3031544960.py:9: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2024 = pd.read_csv(path_2024)


CSV guardado en: /Users/noelp/PycharmProjects/OPTIMET-BCN/data/processed/final_combined_2023_2024.csv


In [9]:
df_total.head()

,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes,tavg,tmin,tmax,prcp,event(y/n),name,attendance
0,2023-01-01,Domingo,1,Internacional,8001,Abrera,8003,Alella,0,11.0,8.6,15.9,0.0,n,NaN,NaN
40110,2023-01-01,Domingo,1,Internacional,8204,Sant Climent de Llobregat,8073,Cornellà de Llobregat,0,11.0,8.6,15.9,0.0,n,NaN,NaN
40111,2023-01-01,Domingo,1,Residente,8204,Sant Climent de Llobregat,8073,Cornellà de Llobregat,0,11.0,8.6,15.9,0.0,n,NaN,NaN
40112,2023-01-01,Domingo,1,Regional,8204,Sant Climent de Llobregat,8074,Cubelles,0,11.0,8.6,15.9,0.0,n,NaN,NaN
40113,2023-01-01,Domingo,1,Regional,8204,Sant Climent de Llobregat,8076,Esparreguera,0,11.0,8.6,15.9,0.0,n,NaN,NaN
